# Exercise: Building and Loading Text Search in Python Whoosh

--- 
<a id='task' ></a>

## Task at hand


For this exercise, we are going to walk through the process of creating full text search capability within Python for integration into other analytical processes.

You previously worked with the _`book`_ data. In this exercise, we will work with some wiki data. 

--- 
<a id='build_it' ></a>

## Buiding our Whoosh Schema

Recall, the `book/` folder is composed of a collection of text files, each its own book chapter.

In whoosh, we structure the retrieval system by defining a storage schema.

From the lab with the text files:
```
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer

schema = Schema(filename=ID(stored=True),
                content=TEXT(analyzer=StemmingAnalyzer())
                )
```

This tells us we are defining records to have a `(filename, content)`

For this exercise, we will be using a few Wikipedia pages for our data source.

### 1) For this exercise, you should look at a few of these web pages:

  * https://en.wikipedia.org/wiki/Nyctimantis
  * https://en.wikipedia.org/wiki/Osteocephalus
  * https://en.wikipedia.org/wiki/Osteopilus
  
Specifically, inspect the HTML source and the 
```HTML
<table class="infobox biota" ... </table>
```



<img src="../images/table_inspect.png" height=400 width=600 />



**Task: You need to extend the above schema definition to collect this frog table data when available.**

* Content will be the all visible text on the html page
* Table information such as kingdom, phylum, class, order, family, subfamily, genus should be searchable 

In [1]:
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer

schema = Schema(filename=ID(stored=True),
                content=TEXT(analyzer=StemmingAnalyzer()),
                # Extend the schema definition to capture relevant table data
                kingdom=KEYWORD(stored=True),
                phylum=KEYWORD(stored=True),
                a_class=KEYWORD(stored=True),
                order=KEYWORD(stored=True),
                family=KEYWORD(stored=True),
                subfamily=KEYWORD(stored=True),
                genus=KEYWORD(stored=True)
               )

--- 
<a id='load_it' ></a>

## Loading Data

For this exercise, we have created a small folder of a few Wikipedia pages under the `en.wikipedia.org/wiki` folder in the common datasets folder:


In [2]:
! ls /dsa/data/all_datasets/en.wikipedia.org/wiki

Acris.html	     Hylidae.html	   Plectrohyla.html
Anotheca.html	     Hylinae.html	   Pseudacris.html
Aparasphenodon.html  Hyloscirtus.html	   Pseudis.html
Aplastodiscus.html   Hypsiboas.html	   Ptychohyla.html
Argenteohyla.html    Isthmohyla.html	   Scarthyla.html
Bokermannohyla.html  Itapotihyla.html	   Scinax.html
Bromeliohyla.html    Lysapsus.html	   Smilisca.html
Charadrahyla.html    Megastomatohyla.html  Sphaenorhynchus.html
Corythomantis.html   Myersiohyla.html	   Tepuihyla.html
Dendropsophus.html   Nyctimantis.html	   Tlalocohyla.html
Duellmanohyla.html   Osteocephalus.html    Trachycephalus.html
Ecnomiohyla.html     Osteopilus.html	   Triprion.html
Exerodonta.html      Phyllodytes.html	   Xenohyla.html
Hyla.html	     Phytotriades.html




You will create the _whoosh_ index files in the `modules/module6/exercises/wiki_index` folder then ingest the files.

To load the data, write a python script that follow the basic crawling behavior

 1. For each file/folder in the specified starting folder:
 1. If it is a folder, recurse into folder and process contents
 1. If it is a file, read contents and load into indexer.
 
## Follow the lab for Python IR with whoosh to complete this exercise.

### 2) Create / Initialize the whoosh index and get the `writer` object.

In [3]:
import os, os.path
from whoosh import index
from bs4 import BeautifulSoup

# Step 2 below this comment"

os.makedirs("animal_index", exist_ok=True)  # create a directory for indexing

# Note, this clears the existing index in the directory
ix = index.create_in("animal_index", schema)

# Get a writer to form the created index in 
writer = ix.writer()


### 3) Adapt the helper functions

Note the subtle changes.
Please adapt the code below such as provided recursive parsing of the HTML (.html) files, indexing with the Whoosh API.
Trust no code, verify all code segments.


In [14]:
def visible(element):  # return those html elements that are visible as text 
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']: #html tags
        return False
    elif re.match('<!--.*-->', str(element)): # html comments
        return False
    return True

def pullBiota(soup):  
        
    data = {}   
   
    table = soup.find('table', class_='infobox')
        
    for row in table.find_all('tr'):
        cells = row.find_all('td')  
        
        
        for i in cells:
            print(i)       
    return data


def loadFile(writer, fname):
    '''
    Read file contents, load into database.
    '''
    with open(fname, 'r') as infile:
        html=infile.read()   # read html content ||||| I CHANGED TO html FROM content
        
        #-----------------------------------------
        soup = BeautifulSoup(html, 'html.parser')
        texts = soup.find_all(text=True)
        
        # Process all the visible text
        visible_texts = filter(visible, texts)
        #-----------------------------------------
        
        print(visible_texts)
        # TODO: Assemble all visible_texts into a content string
        # Hint: Iterate over visible_texts line by line; remove newlines; create a concatenated string
        

        # TODO: Process the "<table class="infobox biota" ... </table> data
        infotable = pullBiota(visible_texts)
        
        writer.add_document(
            filename=fname,
            #content=content,
            kingdom=infotable['Kingdom'],
            phylum=infotable['Phylum'],
            a_class=infotable['Class'],
            order=infotable['Order'],
            family=infotable['Family'],
            subfamily=infotable['Subfamily'],
            genus=infotable['Genus']
        )
        
        # Write to the index
        print("Indexed: ", fname)

def processFolder(writer,folder):
    '''
    Process a folder for files and subfolders
    '''
    print('Processing folder: ',folder)
    for root, dirs, files in os.walk(folder):
        print("root = ", root)
        # Process Files
        for file in files:
            if file.endswith(".html"):
                filename = os.path.join(root, file)
                print('Processing File:',filename)
                loadFile(writer,filename)
            else:
                print("Unhandled File")



# TESTING THE pullBiota() FUNCTION

### 4) Parse with our defined functions in place.

In [15]:
# Start processing the folder and commit the work
# ---------------------------------------------------

processFolder(writer, '/dsa/data/all_datasets/en.wikipedia.org/wiki')
    
writer.commit()


Processing folder:  /dsa/data/all_datasets/en.wikipedia.org/wiki
root =  /dsa/data/all_datasets/en.wikipedia.org/wiki
Processing File: /dsa/data/all_datasets/en.wikipedia.org/wiki/Acris.html


AttributeError: 'filter' object has no attribute 'find'

--- 
<a id='search_me' ></a>

### 5) Executing Queries

Read: 
  http://whoosh.readthedocs.io/en/latest/searching.html
  
Previously, we hard-coded query strings into the code cells.

Now, use the `input()` function collect a query string from the user. 
Then execute the search. For this task, focus only on the `content` field. 

In [ ]:
from whoosh.qparser import QueryParser

# Write your code below this comment:
# --------------------------------------








### 6) Write two example queries to ensure you can search the index 

That is, make sure you can search on the fields you added to the index from the infobox biota table.

```HTML
<table class="infobox biota" ... </table>
```
For this search, we will ignore `content` field and search over the other fields. We can use `MultifieldParser` to specify the fields of our interest. 


In [ ]:
# Write your code below this comment:
# --------------------------------------
from whoosh.qparser import MultifieldParser


# OMIT CONTENT
qp = MultifieldParser(["Kingdom","Phylum","Class","Order","Family","Genus"], 
                      schema=ix.schema, group=qparser.OrGroup)  




In [ ]:
# Write your code below this comment:
# --------------------------------------

# OMIT CONTENT
qp = MultifieldParser(["Kingdom","Phylum","Class","Order","Family","Genus"], 
                      schema=ix.schema, group=qparser.OrGroup)





# SAVE YOUR NOTEBOOK WITH ALL EXECUTED CELLS
# Then, `File > Close and Halt`